In [1]:
import torch
from torch import nn, Tensor
from torch.optim import Optimizer
from torch.utils.data import DataLoader, RandomSampler, DistributedSampler, random_split
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.nn import CrossEntropyLoss
# from pytorch_lightning.core.lightning import LightningModule 
# from pytorch_lightning import LightningModule, Trainer, seed_everything
# from pytorch_lightning.metrics.functional import accuracy, precision, recall
from transformers import AdamW, BertForSequenceClassification, AdamW, BertConfig, AutoTokenizer, BertTokenizer, TrainingArguments
from keras.preprocessing.sequence import pad_sequences


import random
import numpy as np 

import time
import datetime
import pandas as pd
import os
from tqdm import tqdm


import pandas as pd
from transformers import AutoTokenizer, AutoModelWithLMHead
from keras.preprocessing.sequence import pad_sequences

In [2]:
if torch.cuda.is_available():    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: GeForce RTX 2070


In [3]:
pj_path = os.getenv('HOME') + '/Projects'

In [12]:
test_set = pd.read_csv(pj_path + '/crawling_data/google_reviews.csv', index_col=0)
print(len(test_set))
test_set

8408


,ratings,date,label,comment,search,keyword
name,,,,,,
미샤,4.0,2주 전,1.0,여기는 갈 때마다 새로운 느낌이 야 즐거운 시간이 되었습니다,장태산자연휴양림메타세콰이어산림욕장,jangtaesan
Gyang gree young,5.0,3주 전,1.0,하늘 보고 누워 힐링하기 너무 좋은 곳이에요,장태산자연휴양림메타세콰이어산림욕장,jangtaesan
박진수,5.0,3달 전,1.0,잘 정리된 메타세쿼이아 숲에서 산림욕하기 좋은 곳으로 둘레길 걷기와 전망대까지 가벼...,장태산자연휴양림메타세콰이어산림욕장,jangtaesan
Sh Choi,5.0,7달 전,1.0,산책하기 너무 좋은 명소,장태산자연휴양림메타세콰이어산림욕장,jangtaesan
박은선,5.0,1년 전,1.0,주말 마자 애정 하는 장태산 나들이 단풍도 예뻐요,장태산자연휴양림메타세콰이어산림욕장,jangtaesan
...,...,...,...,...,...,...
NaN,NaN,NaN,NaN,NaN,NaN,NaN
NaN,NaN,NaN,NaN,NaN,NaN,NaN
NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
test_set.isna().sum()

ratings      30
date         30
label        30
comment      30
search       30
keyword    2190
dtype: int64

In [14]:
test_set.dropna(inplace=True)

In [15]:
test_set.isna().sum()

ratings    0
date       0
label      0
comment    0
search     0
keyword    0
dtype: int64

In [16]:
len(test_set)

6217

In [18]:
set(test_set.label)

{0.0, 1.0, 2.0}

In [17]:
test_set[test_set.label==0]

,ratings,date,label,comment,search,keyword
name,,,,,,
Woon Kang Paek,5.0,1달 전,0.0,사람 너무 많네,장태산자연휴양림,jangtaesan
jongku LEE,4.0,3달 전,0.0,다 좋았는데 막판에 휴,장태산자연휴양림,jangtaesan
MinJoeng Kim,3.0,3달 전,0.0,생각보다 뭐가 좋은지 모르겠음,장태산자연휴양림,jangtaesan
권수한무와 거북이 삼천갑자 동방삭,4.0,4달 전,0.0,대통령이 갔다 온 곳이라고 해서 기대했는데 무슨 캠핑장에 돌이 많아서 텐트 치는 데...,장태산자연휴양림,jangtaesan
Tom Lee,4.0,4달 전,0.0,나들이 산책 메타세쿼이아 길 구름다리다 있음,장태산자연휴양림,jangtaesan
...,...,...,...,...,...,...
이정석,3.0,1년 전,0.0,애들은 재미있게 잘 놀았지만 시설에 비해 입장료가 비싼 것 같습니다 특히 식당은 메...,오월드,o_world
뭐그냥,5.0,1년 전,0.0,대전사랑 동물과 테마파크,오월드,o_world
jian k,5.0,1년 전,0.0,음,오월드,o_world


In [19]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

tokenizer = AutoTokenizer.from_pretrained("beomi/kcbert-base")


# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
model = BertForSequenceClassification.from_pretrained(
    "beomi/kcbert-base",
    num_labels = 3, 
                    
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initiali

In [20]:
params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The BERT model has 201 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (30000, 768)
bert.embeddings.position_embeddings.weight                (300, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              (

In [21]:
def convert_input_data(sentences):

    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
    MAX_LEN = 64

    # 토큰을 숫자 인덱스로 변환
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
    
    # 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

    # 어텐션 마스크 초기화
    attention_masks = []

    # 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)
        
    inputs = torch.tensor(input_ids)
    masks = torch.tensor(attention_masks)

    return inputs, masks

In [22]:
def test_sentences(sentences):
 
    # 평가모드로 변경!!!!!
    model.eval()

    inputs, masks = convert_input_data(sentences)

    # 데이터를 GPU에 넣음
    b_input_ids = inputs.to(device)
    b_input_mask = masks.to(device)
            
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)

    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()

    return logits

In [23]:
device = "cuda:0"
model = model.to(device)

## 데이터 변환

In [24]:
def preprocessing(df):

    df.document=df.comment.replace('[^A-Za-zㄱ-ㅎㅏ-ㅣ가-힣]+','')

    return df

result = preprocessing(test_set)
result = result.dropna()
print(result)

                  ratings  date  label  \
name                                     
미샤                    4.0  2주 전    1.0   
Gyang gree young      5.0  3주 전    1.0   
박진수                   5.0  3달 전    1.0   
Sh Choi               5.0  7달 전    1.0   
박은선                   5.0  1년 전    1.0   
...                   ...   ...    ...   
young wu Kwon         5.0  1년 전    1.0   
강강수월래                 4.0  1년 전    1.0   
원언니                   5.0  1년 전    1.0   
김경영                   4.0  1년 전    1.0   
김소연                   5.0  1년 전    1.0   

                                                            comment  \
name                                                                  
미샤                                여기는 갈 때마다 새로운 느낌이 야 즐거운 시간이 되었습니다   
Gyang gree young                           하늘 보고 누워 힐링하기 너무 좋은 곳이에요   
박진수               잘 정리된 메타세쿼이아 숲에서 산림욕하기 좋은 곳으로 둘레길 걷기와 전망대까지 가벼...   
Sh Choi                                               산책하기 너무 좋은 명소   
박은선                        

/home/aiffel-dj26/anaconda3/envs/aiffel/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  This is separate from the ipykernel package so we can avoid doing imports until


In [25]:
# 감성분석할 comment 추출 
sens =[]
for sen in result.comment:
    sens.append(sen)
print(len(sens), len(result)) # 개수 확인 
print(sens[1])

6217 6217
하늘 보고 누워 힐링하기 너무 좋은 곳이에요


label : p(1) neg(0) neut(-1)
pred : idx 0, 1, 2 -> 

In [45]:
def make_predicted_label(sen):
    sen = [sen]
    score = test_sentences(sen)
    result = np.argmax(score)

    if result == 0:   # negative 
        return 0
    elif result == 1: # neutral
        return 2
    else:
        return 1   # positive

scores_data=[]
for sen in sens:
    scores_data.append(make_predicted_label(sen))

In [46]:
test_set['pred'] = scores_data

In [47]:
test_set.head()

,ratings,date,label,comment,search,keyword,pred
name,,,,,,,
미샤,4.0,2주 전,1.0,여기는 갈 때마다 새로운 느낌이 야 즐거운 시간이 되었습니다,장태산자연휴양림메타세콰이어산림욕장,jangtaesan,1
Gyang gree young,5.0,3주 전,1.0,하늘 보고 누워 힐링하기 너무 좋은 곳이에요,장태산자연휴양림메타세콰이어산림욕장,jangtaesan,1
박진수,5.0,3달 전,1.0,잘 정리된 메타세쿼이아 숲에서 산림욕하기 좋은 곳으로 둘레길 걷기와 전망대까지 가벼...,장태산자연휴양림메타세콰이어산림욕장,jangtaesan,1
Sh Choi,5.0,7달 전,1.0,산책하기 너무 좋은 명소,장태산자연휴양림메타세콰이어산림욕장,jangtaesan,1
박은선,5.0,1년 전,1.0,주말 마자 애정 하는 장태산 나들이 단풍도 예뻐요,장태산자연휴양림메타세콰이어산림욕장,jangtaesan,1


In [48]:
test_set[test_set.label != test_set.pred]

,ratings,date,label,comment,search,keyword,pred
name,,,,,,,
소병철,4.0,1년 전,1.0,힐링 코스,장태산자연휴양림메타세콰이어산림욕장,jangtaesan,2
Hs jeong,5.0,1년 전,1.0,메타세콰이 아이숲이 울창하고 좋네요 산책하기에도 좋고 담에는 맛있는 음식 가져와 돗...,장태산자연휴양림메타세콰이어산림욕장,jangtaesan,2
Master K,5.0,9시간 전,1.0,장태산은 메타세쿼이아 나무가 높이높이 솟아 있어서 장관입니다 전망대나 흔들 다리도 ...,장태산자연휴양림,jangtaesan,2
Kim Jinwoo,5.0,1주 전,1.0,쥐라기부터 단일 품종으로 생존하고 있는 메타세쿼이아 숲이 주연하는 초록의 향연을 맛...,장태산자연휴양림,jangtaesan,2
dongsek Bae,4.0,3주 전,2.0,자전거 라이딩,장태산자연휴양림,jangtaesan,1
...,...,...,...,...,...,...,...
jian k,5.0,1년 전,0.0,음,오월드,o_world,1
막대사탕,1.0,1년 전,0.0,같이 할 게 없고 바이킹도 재미없다,오월드,o_world,1
21 8,4.0,1년 전,1.0,버드랜드 새 모이주 기 너무 재밌어 오,오월드,o_world,2


In [49]:
set(test_set[test_set.label != test_set.pred].pred)

{0, 1, 2}

In [50]:
set(test_set[test_set.label != test_set.pred].label)

{0.0, 1.0, 2.0}

In [51]:
test_set[test_set.label == test_set.pred]

,ratings,date,label,comment,search,keyword,pred
name,,,,,,,
미샤,4.0,2주 전,1.0,여기는 갈 때마다 새로운 느낌이 야 즐거운 시간이 되었습니다,장태산자연휴양림메타세콰이어산림욕장,jangtaesan,1
Gyang gree young,5.0,3주 전,1.0,하늘 보고 누워 힐링하기 너무 좋은 곳이에요,장태산자연휴양림메타세콰이어산림욕장,jangtaesan,1
박진수,5.0,3달 전,1.0,잘 정리된 메타세쿼이아 숲에서 산림욕하기 좋은 곳으로 둘레길 걷기와 전망대까지 가벼...,장태산자연휴양림메타세콰이어산림욕장,jangtaesan,1
Sh Choi,5.0,7달 전,1.0,산책하기 너무 좋은 명소,장태산자연휴양림메타세콰이어산림욕장,jangtaesan,1
박은선,5.0,1년 전,1.0,주말 마자 애정 하는 장태산 나들이 단풍도 예뻐요,장태산자연휴양림메타세콰이어산림욕장,jangtaesan,1
...,...,...,...,...,...,...,...
이범용,5.0,1년 전,1.0,동물들 많이 있고 놀이기구도 있어서 가족끼리 놀러 오기 좋아요,오월드,o_world,1
young wu Kwon,5.0,1년 전,1.0,재밌어요,오월드,o_world,1
원언니,5.0,1년 전,1.0,초등생 전용 놀이로 아담하고 이 뻐요,오월드,o_world,1


In [52]:
cor = test_set[test_set.label == test_set.pred]
cor[cor.label==1]

,ratings,date,label,comment,search,keyword,pred
name,,,,,,,
미샤,4.0,2주 전,1.0,여기는 갈 때마다 새로운 느낌이 야 즐거운 시간이 되었습니다,장태산자연휴양림메타세콰이어산림욕장,jangtaesan,1
Gyang gree young,5.0,3주 전,1.0,하늘 보고 누워 힐링하기 너무 좋은 곳이에요,장태산자연휴양림메타세콰이어산림욕장,jangtaesan,1
박진수,5.0,3달 전,1.0,잘 정리된 메타세쿼이아 숲에서 산림욕하기 좋은 곳으로 둘레길 걷기와 전망대까지 가벼...,장태산자연휴양림메타세콰이어산림욕장,jangtaesan,1
Sh Choi,5.0,7달 전,1.0,산책하기 너무 좋은 명소,장태산자연휴양림메타세콰이어산림욕장,jangtaesan,1
박은선,5.0,1년 전,1.0,주말 마자 애정 하는 장태산 나들이 단풍도 예뻐요,장태산자연휴양림메타세콰이어산림욕장,jangtaesan,1
...,...,...,...,...,...,...,...
이범용,5.0,1년 전,1.0,동물들 많이 있고 놀이기구도 있어서 가족끼리 놀러 오기 좋아요,오월드,o_world,1
young wu Kwon,5.0,1년 전,1.0,재밌어요,오월드,o_world,1
원언니,5.0,1년 전,1.0,초등생 전용 놀이로 아담하고 이 뻐요,오월드,o_world,1


In [53]:
set(cor.label)

{0.0, 1.0, 2.0}

In [55]:
test_set.to_csv(pj_path + '/sentiment_data/google_pred_kcbert.csv')

## Loss (RMSE)

In [56]:
y = test_set.label
y_pred = test_set.pred
print(len(y), len(y_pred))

6217 6217


In [57]:
type(y), type(y_pred)

(pandas.core.series.Series, pandas.core.series.Series)

In [58]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import math

scaler = MinMaxScaler()

In [59]:
rmse_label = math.sqrt(mean_squared_error(y, y_pred))

In [60]:
print('rmse of label :', rmse_label)

rmse of label : 0.5877961062358544


## Accuracy

In [61]:
if len(y) == len(y_pred):
    total = len(y)
    print(total)
else:
    print('different length')

6217


In [62]:
correct = (y_pred == y).sum().item()

print(f'Accuracy of the network on the {total} test text: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 6217 test text: 72 %


## F-1 score

In [63]:
from sklearn.metrics import f1_score

f1_score(y, y_pred)

ValueError: Target is multiclass but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted'].

In [68]:
def f1_loss(y_true:torch.Tensor, y_pred:torch.Tensor, is_training=False) -> torch.Tensor:
    '''Calculate F1 score. Can work with gpu tensors
    
    The original implmentation is written by Michal Haltuf on Kaggle.
    
    Returns
    -------
    torch.Tensor
        `ndim` == 1. 0 <= val <= 1
    
    Reference
    ---------
    - https://www.kaggle.com/rejpalcz/best-loss-function-for-f1-score-metric
    - https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html#sklearn.metrics.f1_score
    - https://discuss.pytorch.org/t/calculating-precision-recall-and-f1-score-in-case-of-multi-label-classification/28265/6
    
    '''
    assert y_true.ndim == 1
    assert y_pred.ndim == 1 or y_pred.ndim == 2
    
    if y_pred.ndim == 2:
        y_pred = y_pred.argmax(dim=1)
        
    
    tp = (y_true * y_pred).sum().to(torch.float32)
    tn = ((1 - y_true) * (1 - y_pred)).sum().to(torch.float32)
    fp = ((1 - y_true) * y_pred).sum().to(torch.float32)
    fn = (y_true * (1 - y_pred)).sum().to(torch.float32)
    
    epsilon = 1e-7
    
    precision = tp / (tp + fp + epsilon)
    recall = tp / (tp + fn + epsilon)
    
    f1 = 2* (precision*recall) / (precision + recall + epsilon)
    f1.requires_grad = is_training
    return f1

In [69]:
f1_score = f1_loss(y.torch, y_pred)

AttributeError: 'numpy.int64' object has no attribute 'to'

# finetuned with naver movie review dataset(kcbert-base)

In [82]:
test_fine = test_set.copy()

In [74]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

tokenizer = AutoTokenizer.from_pretrained("beomi/kcbert-base")

In [75]:
params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The BERT model has 201 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (30000, 768)
bert.embeddings.position_embeddings.weight                (300, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              (

In [76]:
def convert_input_data(sentences):

    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
    MAX_LEN = 64

    # 토큰을 숫자 인덱스로 변환
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
    
    # 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

    # 어텐션 마스크 초기화
    attention_masks = []

    # 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)
        
    inputs = torch.tensor(input_ids)
    masks = torch.tensor(attention_masks)

    return inputs, masks

In [77]:
def test_sentences(sentences):
 
    # 평가모드로 변경!!!!!
    model.eval()

    inputs, masks = convert_input_data(sentences)

    # 데이터를 GPU에 넣음
    b_input_ids = inputs.to(device)
    b_input_mask = masks.to(device)
            
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)

    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()

    return logits

In [78]:
device = "cuda:0"
model = model.to(device)

## 데이터 변환

In [83]:
def preprocessing(df):

    df.document=df.comment.replace('[^A-Za-zㄱ-ㅎㅏ-ㅣ가-힣]+','')

    return df

result = preprocessing(test_fine)
result = result.dropna()
print(result)

                  name  ratings   date  \
0                 김성주         5  9시간 전   
1                 서알콩         5   1일 전   
2                 윤설희         5   2일 전   
3                 이상훈         4   2일 전   
4               송순이공주         5   2일 전   
..                 ...      ...    ...   
680               김태규         4   4년 전   
681               김뚱이         5   4년 전   
682               이시형         5   4년 전   
683      Park Sangbae         4   4년 전   
684   hyun seung shin         3   4년 전   

                                               comment        search  \
0                                가편하게 산책과 등산을 할 수 있는 곳  계룡산국립공원수통골지구   
1                                   커피 한잔하기 딱 좋은 거리에 용  계룡산국립공원수통골지구   
2                                           비 오고 난 수통골  계룡산국립공원수통골지구   
3                     가족 나들이하기 정말 좋은 곳 국립이라 관리도 잘 돼있네요  계룡산국립공원수통골지구   
4          동네라서 자주 가는데 갈 때마다 좋은 곳이에요 이에는 뱀이 나와서 좀 놀랐어요  계룡산국립공원수통골지구   
..                                                 ...         

/home/aiffel-dj26/anaconda3/envs/aiffel/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  This is separate from the ipykernel package so we can avoid doing imports until


In [84]:
# 감성분석할 comment 추출 
sens =[]
for sen in result.comment:
    sens.append(sen)
print(len(sens), len(result)) # 개수 확인 
print(sens[1])

685 685
커피 한잔하기 딱 좋은 거리에 용


label : p(1) neg(0) neut(-1)
pred : idx 0, 1, 2 -> 

In [109]:
def make_predicted_label(sen):
    sen = [sen]
    score = test_sentences(sen)
    result = np.argmax(score)

    if result == 0:   # negative
        return -1
    elif result == 1: # positive
        return 1


scores_data=[]
for sen in sens:
    scores_data.append(make_predicted_label(sen))

In [110]:
test_fine['pred'] = scores_data

In [111]:
test_fine.head()

,name,ratings,date,comment,search,keyword,label,pred
0,김성주,5,9시간 전,가편하게 산책과 등산을 할 수 있는 곳,계룡산국립공원수통골지구,water_barrel,1,1
1,서알콩,5,1일 전,커피 한잔하기 딱 좋은 거리에 용,계룡산국립공원수통골지구,water_barrel,1,1
2,윤설희,5,2일 전,비 오고 난 수통골,계룡산국립공원수통골지구,water_barrel,1,-1
3,이상훈,4,2일 전,가족 나들이하기 정말 좋은 곳 국립이라 관리도 잘 돼있네요,계룡산국립공원수통골지구,water_barrel,1,1
4,송순이공주,5,2일 전,동네라서 자주 가는데 갈 때마다 좋은 곳이에요 이에는 뱀이 나와서 좀 놀랐어요,계룡산국립공원수통골지구,water_barrel,1,1


In [112]:
test_fine[test_fine.label != test_fine.pred]

,name,ratings,date,comment,search,keyword,label,pred
2,윤설희,5,2일 전,비 오고 난 수통골,계룡산국립공원수통골지구,water_barrel,1,-1
9,Jay Lee,5,1주 전,싱그럽고 일방통행으로 돼 있어 편리해요,계룡산국립공원수통골지구,water_barrel,1,-1
18,은숑,5,2주 전,좋아요 나무 향이요,계룡산국립공원수통골지구,water_barrel,1,-1
21,김태환,5,2주 전,산책로가 잘되어 있어 산책하기 좋습니다 등산로도 있는데 등산하신다면 신발과 배낭 그...,계룡산국립공원수통골지구,water_barrel,-1,1
43,정직한쌤 선생님,5,2달 전,맑은 기 주변 환경 너무 좋아요,계룡산국립공원수통골지구,water_barrel,1,-1
...,...,...,...,...,...,...,...,...
650,동촌,4,2년 전,식당들 음식 가격이 비싸,수통골유원지,water_barrel,1,-1
651,석칠최석칠,5,2년 전,수통골,수통골유원지,water_barrel,1,-1
657,김일환,4,3년 전,산 중턱부터 너무 가파름,수통골유원지,water_barrel,1,-1
660,오션웨이브,5,3년 전,수통골 터줏대감,수통골유원지,water_barrel,1,-1


In [113]:
set(test_fine[test_fine.label != test_fine.pred].pred)

{-1, 1}

In [114]:
set(test_fine[test_fine.label != test_fine.pred].label)

{-1, 0, 1}

In [115]:
test_fine[test_fine.label == test_fine.pred]

,name,ratings,date,comment,search,keyword,label,pred
0,김성주,5,9시간 전,가편하게 산책과 등산을 할 수 있는 곳,계룡산국립공원수통골지구,water_barrel,1,1
1,서알콩,5,1일 전,커피 한잔하기 딱 좋은 거리에 용,계룡산국립공원수통골지구,water_barrel,1,1
3,이상훈,4,2일 전,가족 나들이하기 정말 좋은 곳 국립이라 관리도 잘 돼있네요,계룡산국립공원수통골지구,water_barrel,1,1
4,송순이공주,5,2일 전,동네라서 자주 가는데 갈 때마다 좋은 곳이에요 이에는 뱀이 나와서 좀 놀랐어요,계룡산국립공원수통골지구,water_barrel,1,1
5,김하뇨,5,3일 전,운동을 싫어하는 나는 싫어하지만 아주 좋다 가족이 무척 애용한 다 식당 되게 많고 ...,계룡산국립공원수통골지구,water_barrel,1,1
...,...,...,...,...,...,...,...,...
680,김태규,4,4년 전,완만한 산책 등산 먹거리 좋아요,수통골유원지,water_barrel,1,1
681,김뚱이,5,4년 전,가깝고 깨끗하게 정리도 잘 되어 있어서 좋아요,수통골유원지,water_barrel,1,1
682,이시형,5,4년 전,가끔 산책하러 가는 수통골이에요,수통골유원지,water_barrel,1,1
683,Park Sangbae,4,4년 전,등산을 목적으로 한다면 다녀올만함 여러 코스가 있으니 취향과 체력에 맞춰서 다녀오시...,수통골유원지,water_barrel,-1,-1


In [125]:
cor = test_fine[test_fine.label == test_fine.pred]
cor[cor.label==-1]

,name,ratings,date,comment,search,keyword,label,pred
146,강이돈,2,6달 전,고기 양이 너무 적어 요직 복장이 위생적이지 않음,계룡산국립공원수통골지구,water_barrel,-1,-1
152,Park Tommy,5,6달 전,주차 빼고는 다 좋아요,계룡산국립공원수통골지구,water_barrel,-1,-1
179,유승학,4,7달 전,진입로 협소 주차 간 부족,계룡산국립공원수통골지구,water_barrel,-1,-1
217,이상원,3,8달 전,주말에 갔는데 사람 많아서 그냥 돌아나 옴 깊이 안 가 봐서 모르겠다,계룡산국립공원수통골지구,water_barrel,-1,-1
296,송송승용,2,10달 전,편의시설 부족,계룡산국립공원수통골지구,water_barrel,-1,-1
308,이근영,5,10달 전,갈색 매미충,계룡산국립공원수통골지구,water_barrel,-1,-1
358,Chun-Sil Jin,3,1년 전,초반 쉽다가 돌계단부터 어려움,계룡산국립공원수통골지구,water_barrel,-1,-1
377,조휘탁,4,1년 전,주차가,계룡산국립공원수통골지구,water_barrel,-1,-1
381,EuiMyung Hwang,1,1년 전,물 뜨로 힘들게 갔는데 라돈 검출로 약수 못 뜸,계룡산국립공원수통골지구,water_barrel,-1,-1
437,미식가 딜런,5,3달 전,도시 가까운 곳에 가볍게 망하기 최고 시간이 없어서 계룡산 등산은 아쉽게도 다음 기회로,수통골유원지,water_barrel,-1,-1


In [117]:
set(cor.label)

{-1, 1}

## Loss (RMSE)

In [118]:
y = test_fine.label
y_pred = test_fine.pred
print(len(y), len(y_pred))

685 685


In [119]:
type(y), type(y_pred)

(pandas.core.series.Series, pandas.core.series.Series)

In [120]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import math

scaler = MinMaxScaler()

In [121]:
rmse_label = math.sqrt(mean_squared_error(y, y_pred))

In [122]:
print('rmse of label :', rmse_label)

rmse of label : 0.9065857985976716


## Accuracy

In [123]:
if len(y) == len(y_pred):
    total = len(y)
    print(total)
else:
    print('different length')

685


In [124]:
correct = (y_pred == y).sum().item()

print(f'Accuracy of the network on the {total} test text: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 685 test text: 79 %


## F-1 score

In [106]:
def f1_loss(y_true:torch.Tensor, y_pred:torch.Tensor, is_training=False) -> torch.Tensor:
    '''Calculate F1 score. Can work with gpu tensors
    
    The original implmentation is written by Michal Haltuf on Kaggle.
    
    Returns
    -------
    torch.Tensor
        `ndim` == 1. 0 <= val <= 1
    
    Reference
    ---------
    - https://www.kaggle.com/rejpalcz/best-loss-function-for-f1-score-metric
    - https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html#sklearn.metrics.f1_score
    - https://discuss.pytorch.org/t/calculating-precision-recall-and-f1-score-in-case-of-multi-label-classification/28265/6
    
    '''
    assert y_true.ndim == 1
    assert y_pred.ndim == 1 or y_pred.ndim == 2
    
    if y_pred.ndim == 2:
        y_pred = y_pred.argmax(dim=1)
        
    
    tp = (y_true * y_pred).sum().to(torch.float32)
    tn = ((1 - y_true) * (1 - y_pred)).sum().to(torch.float32)
    fp = ((1 - y_true) * y_pred).sum().to(torch.float32)
    fn = (y_true * (1 - y_pred)).sum().to(torch.float32)
    
    epsilon = 1e-7
    
    precision = tp / (tp + fp + epsilon)
    recall = tp / (tp + fn + epsilon)
    
    f1 = 2* (precision*recall) / (precision + recall + epsilon)
    f1.requires_grad = is_training
    return f1

In [107]:
f1_score = f1_loss(y.torch, y_pred)

AttributeError: 'Series' object has no attribute 'torch'